<a href="https://colab.research.google.com/github/antoniSea/Ethereum-price-prediction/blob/main/Ethereum_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io, os, sys, setuptools, tokenize
!pip install pystan~=2.14
!pip install fbprophet

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
eth_df = yf.download('ETH-USD',start_date, today)
eth_df.tail()

[*********************100%***********************]  1 of 1 completed


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
eth_df.reset_index(inplace=True)
eth_df.columns

df = eth_df[["Date", "Open"]]
new_names = {
    "Date": "ds", 
    "Open": "y",
}
df.rename(columns=new_names, inplace=True)

In [ ]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [ ]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2210,2023-11-28
2211,2023-11-29
2212,2023-11-30
2213,2023-12-01
2214,2023-12-02


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2210,2023-11-28,"$-1,889.56","$-3,643.32",$-123.27
2211,2023-11-29,"$-1,896.30","$-3,701.90",$-85.71
2212,2023-11-30,"$-1,910.41","$-3,774.28",$-81.94
2213,2023-12-01,"$-1,908.98","$-3,699.11",$-60.76
2214,2023-12-02,"$-1,902.44","$-3,725.74",$-93.42


In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()

1142.1377490137004

In [ ]:
 plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)